## Module 6 Homework 

In this homework, we're going to extend Module 5 Homework and learn about streaming with PySpark.

Instead of Kafka, we will use Red Panda, which is a drop-in
replacement for Kafka. 

Ensure you have the following set up (if you had done the previous homework and the module):

- Docker (see [module 1](https://github.com/DataTalksClub/data-engineering-zoomcamp/tree/main/01-docker-terraform))
- PySpark (see [module 5](https://github.com/DataTalksClub/data-engineering-zoomcamp/tree/main/05-batch/setup))

For this homework we will be using the files from Module 5 homework:

- Green 2019-10 data from [here](https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz)


In [1]:
import json
import time
import pandas as pd
import datetime


from kafka import KafkaProducer

In [2]:
def json_serializer(data):
    return json.dumps(data).encode("utf-8")

In [3]:
server = "localhost:9092"

producer = KafkaProducer(bootstrap_servers=[server], value_serializer=json_serializer)

In [4]:
producer.bootstrap_connected()


True

## Question 1: Redpanda version

In [5]:
!docker exec -it redpanda-1 rpk version

v22.3.5 (rev 28b2443)


## Question 2. Creating a topic

In [6]:
!docker exec -it redpanda-1 rpk topic create test-topic

TOPIC       STATUS
test-topic  TOPIC_ALREADY_EXISTS: Topic with this name already exists.


```
TOPIC       STATUS
test-topic  OK
```

## Question 3. Connecting to the Kafka server

In [7]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

## Question 4. Sending data to the stream

In [8]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

producer.flush()

t1 = time.time()    
print(f'took {(t1 - t0):.2f} seconds')


Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
took 0.51 seconds


## Question 5: Sending the Trip Data

In [2]:
!docker exec -it redpanda-1 rpk topic create green-trips

TOPIC        STATUS
green-trips  OK


```
TOPIC        STATUS
green-trips  OK
```

In [ ]:
df_green = pd.read_csv('green_tripdata_2019-10.csv.gz', compression='gzip', low_memory=False)



df_green = df_green[['lpep_pickup_datetime',
                     'lpep_dropoff_datetime',
                     'PULocationID',
                     'DOLocationID',
                     'passenger_count',
                     'trip_distance',
                     'tip_amount']]

In [11]:
topic_name = 'green-trips'
t0 = time.time()
for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    row_dict['timetstamp'] = datetime.datetime.now().timestamp()
    producer.send(topic_name, value=row_dict)
t1 = time.time()

print(f'It took {round(t1 - t0, 1)} seconds to send these messages.')


It took 63.1 seconds to send these messages.


In [3]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

:: loading settings :: url = jar:file:/home/marfanyan/spark/spark-3.5.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/marfanyan/.ivy2/cache
The jars for the packages stored in: /home/marfanyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-46903e3e-65b8-4727-b68c-cb5eac50f84d;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 1231ms :: artifacts dl 5

In [4]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [14]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

24/03/16 06:38:45 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-1987d72d-ba6c-438b-ab64-1e9b46530553. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/16 06:38:45 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/16 06:38:47 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


In [15]:
query.stop()


24/03/16 06:38:51 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (de-zoomcamp.europe-west1-b.c.ny-rides-marfanyan.internal executor driver): TaskKilled (Stage cancelled: Job 0 cancelled part of cancelled job group 5d86e8fa-c511-4e78-92a9-8810b3f9f9dc)


## Question 6. Parsing the data


In [5]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [6]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [7]:
green_stream.printSchema()


root
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- tip_amount: double (nullable = true)



## Question 7: Most popular destination

In [8]:
green_stream = green_stream.withColumn("timestamp", F.current_timestamp())




popular_destinations = green_stream.groupBy(
    F.window(
        green_stream.timestamp,
        windowDuration='5 minutes'
    ),
    green_stream.DOLocationID,
).count().orderBy(F.desc('count'))

In [ ]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()
query.stop()

24/03/16 06:56:36 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-9ba80686-f698-4e57-85f0-eeada4c37eb2. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/16 06:56:36 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/16 06:56:37 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+------------+-----+
|window|DOLocationID|count|
+------+------------+-----+
+------+------------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|74          |7564 |
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|42          |6708 |
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|41          |6148 |
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|75          |5343 |
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|7           |5065 |
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|129         |4990 |
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|166         |4838 |
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|236         |3422 |
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|238         |3197 |
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|223         |3197 |
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|82          |3122 |
|{2024-

-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|74          |16717|
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|42          |14671|
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|41          |13374|
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|75          |11870|
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|129         |11297|
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|7           |10932|
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|166         |10503|
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|236         |7584 |
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|223         |7260 |
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|238         |7001 |
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|82          |6706 |
|{2024-

-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|74          |17270|
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|42          |15353|
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|41          |13754|
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|75          |12428|
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|129         |11671|
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|7           |11277|
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|166         |10668|
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|236         |7782 |
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|223         |7436 |
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|238         |7184 |
|{2024-03-16 06:55:00, 2024-03-16 07:00:00}|82          |7021 |
|{2024-